Uses the files created using the scripts in `data` and sets up the whole training script.

In [83]:
import pandas as pd
import numpy as np
import os
import glob
import shutil

In [113]:
runs = ['mw_2016','nigeria_2013','uganda_2011','tanzania_2011']
run = runs[2]
print(run)

uganda_2011


In [132]:
df_download = pd.read_csv('data/output/{}_download.csv'.format(run))
df_download_info = pd.read_csv('data/output/{}_download_info.csv'.format(run))
df_guide = pd.read_csv('data/output/{}_guide.csv'.format(run))

In [133]:
files = os.listdir('data/ims_{}'.format(run))
df_download_info['check'] = df_download_info.images.apply(lambda x: (x in files))
print(len(df_download_info)-df_download_info.check.sum())
df_download_info.head()

0


,im_lat,im_lon,clust_lat,clust_lon,nightlight,consumption,nightlight_bin,images,check
0,-1.125000,29.574999,-1.150315,29.591431,0.0,2.033755,1,-1.125_29.574999.png,True
1,-1.116666,29.591666,-1.150315,29.591431,0.0,2.033755,1,-1.116666_29.591665999999996.png,True
2,-1.133333,29.591666,-1.150315,29.591431,0.0,2.033755,1,-1.133333_29.591665999999996.png,True
3,-1.150000,29.599999,-1.150315,29.591431,0.0,2.033755,1,-1.15_29.599999.png,True
4,-1.158333,29.608332,-1.150315,29.591431,0.0,2.033755,1,-1.158333_29.608332.png,True


In [134]:
try:
    assert len(df_download) == len(df_download_info)
    # this means the script broke and you restarted it
    # this is how I named images so it should work...
#     This is wrong somehow 
#     df_download['images'] = df_download.apply(lambda x: str(x.im_lat) + '_' + str(x.im_lon) + '.png', axis=1)
except:
    print('broken')
    print(len(df_download))
    print(len(df_download_info))
    df_download = df_download_info

In [136]:
# we didn't download duplicate images which explains the difference
df_download.shape, df_guide.shape

((25246, 9), (33880, 7))

In [137]:
len(df_guide.groupby(['clust_lat', 'clust_lon'])) # number of clusters

636

In [138]:
# merges the images downloaded with the original dataframe
df_sub = df_download[['im_lat', 'im_lon', 'images']]
df = pd.merge(left=df_guide, right=df_sub, on=['im_lat', 'im_lon'])

In [139]:
# I didn't download all the images and also I got rid of some image repeats, hence the difference
df_guide.shape, df.shape

((33880, 7), (33880, 8))

In [140]:
# adds a "cluster number" to the dataframe
clust_group = df.groupby(['clust_lat', 'clust_lon'])
clust_group = clust_group.first().reset_index()[['clust_lat', 'clust_lon']]
clust_numbers = np.arange(len(clust_group))
clust_group['clust_num'] = clust_numbers
df = pd.merge(df, clust_group, on=['clust_lat', 'clust_lon'])
df.head()

,im_lat,im_lon,clust_lat,clust_lon,nightlight,consumption,nightlight_bin,images,clust_num
0,-1.125000,29.574999,-1.150315,29.591431,0.0,2.033755,1,-1.125_29.574999.png,11
1,-1.116666,29.591666,-1.150315,29.591431,0.0,2.033755,1,-1.116666_29.591665999999996.png,11
2,-1.133333,29.591666,-1.150315,29.591431,0.0,2.033755,1,-1.133333_29.591665999999996.png,11
3,-1.150000,29.599999,-1.150315,29.591431,0.0,2.033755,1,-1.15_29.599999.png,11
4,-1.158333,29.608332,-1.150315,29.591431,0.0,2.033755,1,-1.158333_29.608332.png,11


I'm gonna add an _ to the images to show which cluster they come from

In [141]:
df['images_renamed'] = df.apply(lambda x: x.images[:-4] + '_' + str(x.clust_num) + '.png', axis=1)
df.to_csv('data/output/{}_full_guide.csv'.format(run), index=False)
df.head()

,im_lat,im_lon,clust_lat,clust_lon,nightlight,consumption,nightlight_bin,images,clust_num,images_renamed
0,-1.125000,29.574999,-1.150315,29.591431,0.0,2.033755,1,-1.125_29.574999.png,11,-1.125_29.574999_11.png
1,-1.116666,29.591666,-1.150315,29.591431,0.0,2.033755,1,-1.116666_29.591665999999996.png,11,-1.116666_29.591665999999996_11.png
2,-1.133333,29.591666,-1.150315,29.591431,0.0,2.033755,1,-1.133333_29.591665999999996.png,11,-1.133333_29.591665999999996_11.png
3,-1.150000,29.599999,-1.150315,29.591431,0.0,2.033755,1,-1.15_29.599999.png,11,-1.15_29.599999_11.png
4,-1.158333,29.608332,-1.150315,29.591431,0.0,2.033755,1,-1.158333_29.608332.png,11,-1.158333_29.608332_11.png


In [123]:
# EDIT THIS OUT to keep only one copy of file in ims, no neeed for original renamed because shututil takes a long timeeeee
# you can add other countries here under the structure images/<COUNTRY>/
# os.makedirs('../images/ims_{}/ims'.format(run))

# # this will copy images into a folder called ims
# # this folder is helpful because the original folder has all unique images
# # now, we need to duplicate those images and distinguish them by their new name (as made previously)
# def create_im_renamed(x):
#     shutil.copy('data/ims_{}/{}'.format(run, x.images), '../images/ims_{}/ims/{}'.format(run, x.images_renamed))        
# df.apply(create_im_renamed,axis=1)

Create train/valid folders

The model will use these directly.

In [142]:
pic_list = df['images'].values.tolist()
to_pick = int(0.8*len(pic_list)); to_pick

27104

In [143]:
inds = np.arange(len(pic_list))
train_ind = np.random.choice(np.arange(len(pic_list)), to_pick, replace=False)
valid_ind = np.delete(inds, train_ind)

In [144]:
pic_list = np.array(pic_list)
train_im = pic_list[train_ind]
valid_im = pic_list[valid_ind]

In [145]:
os.makedirs('../images/ims_{}/train'.format(run), exist_ok=False)
os.makedirs('../images/ims_{}/valid'.format(run), exist_ok=False)

FileExistsError: [Errno 17] File exists: '../images/ims_uganda_2011/train'

In [146]:
t = df.iloc[train_ind]
v = df.iloc[valid_ind]

In [148]:
for fi, fn, l in zip(t['images'], t['images_renamed'], t['nightlight_bin']):
    os.makedirs('../images/ims_{}/train/{}'.format(run, l), exist_ok=True)
    shutil.copy('data/ims_{}/{}'.format(run, fi), '../images/ims_{}/train/{}/{}'.format(run, l, fn))
for fi,fn, l in zip(v['images'],v['images_renamed'], v['nightlight_bin']):
    os.makedirs('../images/ims_{}/valid/{}'.format(run, l), exist_ok=True)
    shutil.copy('data/ims_{}/{}'.format(run, fi), '../images/ims_{}/valid/{}/{}'.format(run, l, fn))

In [149]:
# shows count distribution in train folder
for i in range(1,4):
    print(len(os.listdir('../images/ims_{}/train/{}'.format(run,str(i)))))

11287
7632
8185


In [150]:
# shows count distribution in valid folder
for i in range(1,4):
    print(len(os.listdir('../images/ims_{}/valid/{}'.format(run, str(i)))))

2836
1923
2017
